In [1]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

In [10]:

def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

def logistic(x,r,n=1):
    if n==1: return r*x*(1-x)
    return logistic(logistic(x,r,n-1),r,1)

class logi(widgets.HBox):
     
    def __init__(self):
        super().__init__()
        output = widgets.Output()
 
        self.x = np.linspace(0, 1, 300)
        #initial_color = '#FF00DD'
        self.f=logistic
        with output:
            self.fig, self.ax = plt.subplots(constrained_layout=True, figsize=(5, 3.5))
            self.ax.set_xlim(0,1)
            self.ax.set_ylim(0,1)
        self.ax.set_xlabel(r'$x_n$')
        self.ax.set_ylabel(r'$x_{n+1}$')

        self.curve, = self.ax.plot(self.x, self.f(self.x,2))
        self.cobweb,= self.ax.plot([0],[0])
        self.bitect,= self.ax.plot([0,1],[0,1])


        self.fig.canvas.toolbar_position = 'bottom'
        self.nIterMax=300
        #self.ax.grid(True)
 
        # define widgets
        r_slider = widgets.FloatSlider(
            value=2, 
            min=0, 
            max=4,
            step=1e-2, 
            continuous_update=True,
            description='r'
        )
        l_slider= widgets.FloatLogSlider(
            value=0, 
            min=-2.5, 
            max=2.5, 
            continuous_update=True,
            description='l'
        )
        x_slider= widgets.FloatSlider(
            value=0.4, 
            min=0, 
            max=1,
            step=1e-2, 
            continuous_update=True,
            description='x'
        )
        n_slider= widgets.IntSlider(
            value=1, 
            min=1, 
            max=8,
            step=1, 
            continuous_update=True,
            description='n'
        )
        b_checkbox= widgets.Checkbox(
            value=True,
            description='bisettrice',
            disabled=False
        )
 
        controls = widgets.VBox([
            r_slider, 
            l_slider,
            x_slider,
            n_slider,
            b_checkbox
        ])
        controls.layout = make_box_layout()
         
        out_box = widgets.Box([output])
        output.layout = make_box_layout()
 
        # observe stuff
        widgets.interactive(self.updateCobwebs,r=r_slider, x=x_slider, l=l_slider, n=n_slider ,continuous_update=True)
        widgets.interactive(self.updateCurve,r=r_slider,n=n_slider)
        widgets.interactive(self.update_ylabel,n=n_slider)
        widgets.interactive(self.updateBisec,b=b_checkbox)
        
         
 
        # add to children
        self.children = [controls, output]

     
    def updateCurve(self, r, n):
        """Draw line in plot"""
        self.curve.set_ydata(self.f(self.x,r,n))
        self.fig.canvas.draw()
    
    def updateCobwebs(self,r,x,l,n):
        vertex=[[x],[x]]
        for i in range(self.nIterMax):  
            xp1=self.f(x,r,n)
            if l/np.abs(xp1-x)<1:
                vertex[0]+=[x]
                vertex[1]+=[x+l*np.sign(xp1-x)]
                break 
            if l/np.abs(xp1-x)<2:
                vertex[0]+=[x,x+l*np.sign(xp1-x)/2]
                vertex[1]+=[xp1,xp1]
                break
            
            vertex[0]+=[x,xp1]
            vertex[1]+=[xp1,xp1]
            l-=2*np.abs(x-xp1)
            x=xp1

        self.cobweb.set_xdata(vertex[0])
        self.cobweb.set_ydata(vertex[1])
        self.fig.canvas.draw()
 
    def update_ylabel(self, n):
        self.ax.set_ylabel(r'$x_{n+%d}$'%(n))

    def updateBisec(self,b):
        if b==False:
            self.bitect.set_xdata([0])
            self.bitect.set_ydata([0])
        else:
            self.bitect.set_xdata([0,1])
            self.bitect.set_ydata([0,1])
        self.fig.canvas.draw()
    
         
         
logi()

logi(children=(VBox(children=(FloatSlider(value=2.0, description='r', max=4.0, step=0.01), FloatLogSlider(valu…